# oanda3通貨アビトラ

In [1]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_finance
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.endpoints.pricing import PricingInfo
from oandapyV20.contrib.requests import (MarketOrderRequest, StopLossDetails)
import oandapyV20.endpoints.forexlabs as labs
import datetime
from statistics import mean
from statistics import median
import calendar
import datetime
from math import floor
import time
import itertools

In [2]:
# OANDA API v20の口座IDとAPIトークン(藤沼ver)
accountID = "101-009-12789646-001"
access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")

In [3]:
# APIから取得したレートをPandasのDataFrameへ
def to_dataframe(r):
    data = []
    for raw in r.response['candles']:
        data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])

    # リストからPandas DataFrameへ変換
    df = pd.DataFrame(data)
    df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
    df['Time'] = pd.to_datetime(df['Time'])
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].apply(float)
    return df

In [4]:
def get_data(count, gran, year, month, day, hour, minute, second):
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    _from = datetime.datetime(year=year, month=month, day=day,
                              hour=hour, minute=minute, second=second).strftime(fmt)
    params = {
        "count": count,
        "granularity": gran,
        'from': _from,
#         'dailyAlignment': 0
    }
    r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
    api.request(r)
    df = to_dataframe(r)
    return df

In [5]:
# 第何何曜日（第nX曜日）の日付を返す関数
def get_day_of_nth_dow(year, month, nth, dow):
    '''dow: Monday(0) - Sunday(6)'''
    if nth < 1 or dow < 0 or dow > 6:
        return None

    first_dow, n = calendar.monthrange(year, month)
    day = 7 * (nth - 1) + (dow - first_dow) % 7 + 1

    return day if day <= n else None

In [6]:
def get_data_super(start, end, gran, inst='USD_JPY'):
    f = True
    f_count = 0
    while f:
        try:
            # OANDA API v20の口座IDとAPIトークン
            accountID = "101-009-12609641-001"
            access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
            # OANDAのデモ口座へのAPI接続
            api = API(access_token=access_token, environment="practice")
            # xxxx-xx-xx～xxxx-xx-xx の期間のデータを取得してデータフレームを返す
            s_year, s_month, s_day, s_hour, s_minute, s_second = map(int, start.split('-'))
            e_year, e_month, e_day, e_hour, e_minute, e_second = map(int, end.split('-'))
            fmt = '%Y-%m-%dT%H:%M:00.000000Z'
            from_ = datetime.datetime(year=s_year, month=s_month, day=s_day,
                                      hour=s_hour, minute=s_minute, second=s_second).strftime(fmt)
            to_ = datetime.datetime(year=e_year, month=e_month, day=e_day,
                                    hour=e_hour, minute=e_minute, second=e_second).strftime(fmt)
            to_dt = '{}-{}-{} {}:{}:{}'.format(e_year, e_month, e_day, e_hour, e_minute, e_second)
            to_unix = datetime.datetime.strptime(to_dt, '%Y-%m-%d %H:%M:%S').timestamp()
            df = pd.DataFrame()
            # 5000個制限に引っかからなければこっちの処理
            try:
                params = {
                    "granularity": gran,
                    'from': from_,
                    'to': to_
                }
                r = instruments.InstrumentsCandles(instrument=inst, params=params)
                api.request(r)
                df = to_dataframe(r)
                f = False
            # 引っかかればこっちの処理
            except:
                i = True
                while i:
                    df_tmp = get_data(count=5000, gran=gran, year=s_year, month=s_month, day=s_day,
                                     hour=s_hour, minute=s_minute, second=s_second, inst=inst)
                    # ケツが同じなら止める (最新迄用)
                    if df.tail(1).reset_index(drop=True).equals(df_tmp.tail(1).reset_index(drop=True)):
                        i = False
                    else:
                        df = pd.concat([df, df_tmp[:-1]], ignore_index=True)
                        index = df_tmp.tail(1).index[0]
                        s_year = df_tmp.iat[index, 0].year
                        s_month = df_tmp.iat[index, 0].month
                        s_day = df_tmp.iat[index, 0].day
                        s_hour = df_tmp.iat[index, 0].hour
                        s_minute = df_tmp.iat[index, 0].minute
                        s_second = df_tmp.iat[index, 0].second
                        # 5000個ずつ取得してエンドの unix 超えたら止める
                        if to_unix < df.iat[df.tail(1).index[0], 0].timestamp():
                            # はみ出した分をここで切り落とす
                            for i in range(len(df.index) - 5000, len(df.index)):
                                if to_unix <= df.iat[i, 0].timestamp():
                                    df = df[:i]
                                    break
                            i = False
                f = False
        except Exception as e:
            if f_count > 10:
                print(e)
                f = False
            time.sleep(5)
            f_count+=1
            pass
    return df

## abitrage

In [26]:
params = {"instruments": "EUR_USD"}
pi = PricingInfo(accountID=accountID, params=params)
r = api.request(pi)
EUR_USD = float(r['prices'][0]['asks'][0]['price'])
params = {"instruments": "EUR_GBP"}
pi = PricingInfo(accountID=accountID, params=params)
r = api.request(pi)
EUR_GBP = float(r['prices'][0]['bids'][0]['price'])
params = {"instruments": "GBP_USD"}
pi = PricingInfo(accountID=accountID, params=params)
r = api.request(pi)
GBP_USD = float(r['prices'][0]['bids'][0]['price'])
1 * EUR_USD / EUR_GBP / GBP_USD

1.0001683968249833

In [8]:
EUR_USD, EUR_GBP, GBP_USD

(1.11327, 0.85796, 1.29738)

## 一番歪みの大きいペアを探す

In [8]:
# 取引可能ペアを2次元配列で返す
def instruments():
    # OANDA API v20の口座IDとAPIトークン(藤沼ver)
    accountID = "101-009-12789646-001"
    access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    r = accounts.AccountInstruments(accountID=accountID)
    rv = api.request(r)
    inst_l = [d['name'] for d in r.response['instruments']]
    inst_ll = []
    for i in inst_l:
        inst_ll.append(i.split('_'))
    return inst_ll

In [9]:
# スプレッド取得
def pip_spread(instrument, print_=False):
    # OANDA API v20の口座IDとAPIトークン(藤沼ver)
    accountID = "101-009-12789646-001"
    access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    pip = 0
    r = accounts.AccountInstruments(accountID=accountID)
    api.request(r)
    for i in r.response['instruments']:
        if i['name'] == instrument:
            pip = 10**i['pipLocation']
    spread = 0
    r = pricing.PricingInfo(accountID=accountID, params={'instruments': instrument})
    api.request(r)
    response = r.response
    spread = float(response['prices'][0]['asks'][0]['price'])-float(response['prices'][0]['bids'][0]['price'])
    spread = round(spread / pip, 1)
    if print_:
        root = instrument.split('_')[1]
        print('{}{} = 1pips, spread: {}pips, {}{}'.format(pip, root, spread, pip*spread, root))
    else:
        return pip, spread, pip*spread

In [10]:
# 現在の価格を、タプル形式で取得
def get_price(instrument):
    # OANDA API v20の口座IDとAPIトークン(藤沼ver)
    accountID = "101-009-12789646-001"
    access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    r = pricing.PricingInfo(accountID=accountID, params={'instruments': instrument})
    rv = api.request(r)
    return {'buy': float(rv['prices'][0]['asks'][0]['price']),
            'sell': float(rv['prices'][0]['bids'][0]['price'])}

In [11]:
pip_spread('USD_THB', print_=True)

0.01THB = 1pips, spread: 8.7pips, 0.087THB


In [92]:
price = get_price('USD_JPY')
price['buy']

109.924

In [44]:
# 全通貨を取得、組み合わせを算出
inst_l = instruments()
all_inst = []
for i in inst_l:
    for j in i:
        if not j in all_inst:
            all_inst.append(j)
ls = list(itertools.permutations(list(range(len(all_inst))), 3))

In [47]:
len(all_inst)

23

In [45]:
len(ls)

10626

In [282]:
T0_l = []
T1_l = []
T2_l = []
side1_l = []
side2_l = []
side3_l = []
prof_l = []
for l in ls:
    T0, T1, T2 = all_inst[l[0]], all_inst[l[1]], all_inst[l[2]]
#     T0, T1, T2 = 'USD', 'EUR', 'GBP'
    # buy T1 / T0
    try:
        try:
            buyT1 = 1 / get_price('{}_{}'.format(T1, T0))['buy']
            side1 = 'buy'
        except V20Error:
            buyT1 = get_price('{}_{}'.format(T0, T1))['sell']
            side1 = 'sell'
    except V20Error:
        continue
    
    # buy T2/ T1
    try:
        try:
            buyT2 = buyT1 / get_price('{}_{}'.format(T2, T1))['buy']
            side2 = 'buy'
        except V20Error:
            buyT2 = buyT1 * get_price('{}_{}'.format(T1, T2))['sell']
            side2 = 'sell'
    except V20Error:
        continue
    
    # buy T0 / T2
    try:
        try:
            buyT0 = buyT2 / get_price('{}_{}'.format(T0, T2))['buy'] - 1
            side3 = 'buy'
        except V20Error:
            buyT0 = buyT2 * get_price('{}_{}'.format(T2, T0))['sell'] - 1
            side3 = 'sell'
    except V20Error:
        continue
    
    profit = buyT0
    print('T0: {}, T1: {}, T2: {}, profit: {:>13.10f}'.format(T0, T1, T2, profit))
    T0_l.extend([T0])
    T1_l.extend([T1])
    T2_l.extend([T2])
    side1_l.extend([side1])
    side2_l.extend([side2])
    side3_l.extend([side3])
    prof_l.extend([profit])
abiPair = pd.DataFrame(data={'T0': T0_l, 'T1': T1_l, 'T2': T2_l,
                             'side1': side1_l,'side2': side2_l,'side3': side3_l, 'profit(T0)': prof_l},
                       columns=["T0", "T1", "T2", 'side1', 'side2', 'side3', "profit(T0)"])

T0: USD, T1: ZAR, T2: EUR, profit: -0.0011570122
T0: USD, T1: ZAR, T2: GBP, profit: -0.0011811757
T0: USD, T1: ZAR, T2: SGD, profit: -0.0011025809
T0: USD, T1: ZAR, T2: HKD, profit: -0.0010560187
T0: USD, T1: ZAR, T2: JPY, profit: -0.0011879265
T0: USD, T1: ZAR, T2: CAD, profit: -0.0011137460
T0: USD, T1: ZAR, T2: CHF, profit: -0.0011695957
T0: USD, T1: ZAR, T2: AUD, profit: -0.0011550032
T0: USD, T1: EUR, T2: ZAR, profit: -0.0009601913
T0: USD, T1: EUR, T2: GBP, profit: -0.0001630230
T0: USD, T1: EUR, T2: SEK, profit: -0.0005100063
T0: USD, T1: EUR, T2: SGD, profit: -0.0002004919
T0: USD, T1: EUR, T2: HKD, profit: -0.0001246307
T0: USD, T1: EUR, T2: JPY, profit: -0.0000870377
T0: USD, T1: EUR, T2: NZD, profit: -0.0002242343
T0: USD, T1: EUR, T2: CAD, profit: -0.0001683485
T0: USD, T1: EUR, T2: CHF, profit: -0.0001986924
T0: USD, T1: EUR, T2: PLN, profit: -0.0010073049
T0: USD, T1: EUR, T2: CNH, profit: -0.0002185682
T0: USD, T1: EUR, T2: HUF, profit: -0.0020214014
T0: USD, T1: EUR, T2

T0: USD, T1: CHF, T2: NOK, profit: -0.0006586685
T0: USD, T1: CHF, T2: CZK, profit: -0.0012835189
T0: USD, T1: CHF, T2: MXN, profit: -0.0004540441
T0: USD, T1: PLN, T2: EUR, profit: -0.0009429609
T0: USD, T1: PLN, T2: GBP, profit: -0.0010654185
T0: USD, T1: PLN, T2: SGD, profit: -0.0010031243
T0: USD, T1: PLN, T2: HKD, profit: -0.0009335346
T0: USD, T1: PLN, T2: JPY, profit: -0.0010918417
T0: USD, T1: PLN, T2: CAD, profit: -0.0013644710
T0: USD, T1: PLN, T2: CHF, profit: -0.0013354610
T0: USD, T1: PLN, T2: AUD, profit: -0.0010374856
T0: USD, T1: CNH, T2: EUR, profit: -0.0002206874
T0: USD, T1: CNH, T2: GBP, profit: -0.0002686399
T0: USD, T1: CNH, T2: SGD, profit: -0.0002603938
T0: USD, T1: CNH, T2: HKD, profit: -0.0002035998
T0: USD, T1: CNH, T2: JPY, profit: -0.0038267859
T0: USD, T1: CNH, T2: CAD, profit: -0.0002926121
T0: USD, T1: CNH, T2: CHF, profit: -0.0003229361
T0: USD, T1: CNH, T2: AUD, profit: -0.0003262346
T0: USD, T1: HUF, T2: EUR, profit: -0.0018755849
T0: USD, T1: HUF, T2

T0: ZAR, T1: CHF, T2: USD, profit: -0.0014213959
T0: ZAR, T1: CHF, T2: EUR, profit: -0.0014652399
T0: ZAR, T1: CHF, T2: GBP, profit: -0.0015224640
T0: ZAR, T1: CHF, T2: SGD, profit: -0.0015353845
T0: ZAR, T1: CHF, T2: HKD, profit: -0.0013745853
T0: ZAR, T1: CHF, T2: JPY, profit: -0.0016377937
T0: ZAR, T1: CHF, T2: CAD, profit: -0.0015326816
T0: ZAR, T1: CHF, T2: AUD, profit: -0.0015811553
T0: ZAR, T1: AUD, T2: USD, profit: -0.0014540607
T0: ZAR, T1: AUD, T2: EUR, profit: -0.0014860092
T0: ZAR, T1: AUD, T2: GBP, profit: -0.0015350819
T0: ZAR, T1: AUD, T2: SGD, profit: -0.0015907063
T0: ZAR, T1: AUD, T2: HKD, profit: -0.0015246695
T0: ZAR, T1: AUD, T2: JPY, profit: -0.0014582844
T0: ZAR, T1: AUD, T2: CAD, profit: -0.0015614171
T0: ZAR, T1: AUD, T2: CHF, profit: -0.0016157192
T0: EUR, T1: USD, T2: ZAR, profit: -0.0012289561
T0: EUR, T1: USD, T2: GBP, profit: -0.0001654081
T0: EUR, T1: USD, T2: SEK, profit: -0.0004246904
T0: EUR, T1: USD, T2: SGD, profit: -0.0001881765
T0: EUR, T1: USD, T2

T0: EUR, T1: CHF, T2: GBP, profit: -0.0003213442
T0: EUR, T1: CHF, T2: SEK, profit: -0.0005583886
T0: EUR, T1: CHF, T2: SGD, profit: -0.0003784026
T0: EUR, T1: CHF, T2: HKD, profit: -0.0003224877
T0: EUR, T1: CHF, T2: JPY, profit: -0.0001982043
T0: EUR, T1: CHF, T2: NZD, profit: -0.0004252262
T0: EUR, T1: CHF, T2: CAD, profit: -0.0003713653
T0: EUR, T1: CHF, T2: PLN, profit: -0.0011686207
T0: EUR, T1: CHF, T2: CNH, profit: -0.0003959172
T0: EUR, T1: CHF, T2: HUF, profit: -0.0020106623
T0: EUR, T1: CHF, T2: AUD, profit: -0.0003526524
T0: EUR, T1: CHF, T2: SAR, profit: -0.0008196177
T0: EUR, T1: CHF, T2: THB, profit: -0.0010762314
T0: EUR, T1: CHF, T2: TRY, profit: -0.0014272821
T0: EUR, T1: CHF, T2: INR, profit: -0.0010347021
T0: EUR, T1: CHF, T2: DKK, profit: -0.0003551573
T0: EUR, T1: CHF, T2: NOK, profit: -0.0006715056
T0: EUR, T1: CHF, T2: CZK, profit: -0.0011900094
T0: EUR, T1: CHF, T2: MXN, profit: -0.0005903245
T0: EUR, T1: PLN, T2: USD, profit: -0.0008490620
T0: EUR, T1: PLN, T2

T0: GBP, T1: EUR, T2: HUF, profit: -0.0021372037
T0: GBP, T1: EUR, T2: AUD, profit: -0.0002939895
T0: GBP, T1: EUR, T2: SAR, profit: -0.0008026834
T0: GBP, T1: EUR, T2: THB, profit: -0.0010311317
T0: GBP, T1: EUR, T2: TRY, profit: -0.0012169493
T0: GBP, T1: EUR, T2: INR, profit: -0.0010288777
T0: GBP, T1: EUR, T2: DKK, profit: -0.0003795454
T0: GBP, T1: EUR, T2: NOK, profit: -0.0007561884
T0: GBP, T1: EUR, T2: CZK, profit: -0.0011916094
T0: GBP, T1: EUR, T2: MXN, profit: -0.0005875684
T0: GBP, T1: SEK, T2: USD, profit: -0.0005162332
T0: GBP, T1: SEK, T2: EUR, profit: -0.0005732858
T0: GBP, T1: SEK, T2: SGD, profit: -0.0006878247
T0: GBP, T1: SEK, T2: HKD, profit: -0.0006439734
T0: GBP, T1: SEK, T2: JPY, profit: -0.0006413620
T0: GBP, T1: SEK, T2: CAD, profit: -0.0006686656
T0: GBP, T1: SEK, T2: CHF, profit: -0.0006537110
T0: GBP, T1: SEK, T2: AUD, profit: -0.0007041037
T0: GBP, T1: SGD, T2: USD, profit: -0.0002267403
T0: GBP, T1: SGD, T2: ZAR, profit: -0.0014573069
T0: GBP, T1: SGD, T2

...

T0: HKD, T1: DKK, T2: USD, profit: -0.0002918461
T0: HKD, T1: DKK, T2: EUR, profit: -0.0003329825
T0: HKD, T1: DKK, T2: GBP, profit: -0.0004058455
T0: HKD, T1: DKK, T2: SGD, profit: -0.0003916425
T0: HKD, T1: DKK, T2: JPY, profit: -0.0003915126
T0: HKD, T1: DKK, T2: CAD, profit: -0.0004120364
T0: HKD, T1: DKK, T2: CHF, profit: -0.0004428204
T0: HKD, T1: DKK, T2: AUD, profit: -0.0005288903
T0: HKD, T1: NOK, T2: USD, profit: -0.0004464493
T0: HKD, T1: NOK, T2: EUR, profit: -0.0004859188
T0: HKD, T1: NOK, T2: GBP, profit: -0.0006202762
T0: HKD, T1: NOK, T2: SGD, profit: -0.0005510736
T0: HKD, T1: NOK, T2: JPY, profit: -0.0005356614
T0: HKD, T1: NOK, T2: CAD, profit: -0.0005662195
T0: HKD, T1: NOK, T2: CHF, profit: -0.0006361082
T0: HKD, T1: NOK, T2: AUD, profit: -0.0005655015
T0: HKD, T1: CZK, T2: USD, profit: -0.0007241528
T0: HKD, T1: CZK, T2: EUR, profit: -0.0009258303
T0: HKD, T1: CZK, T2: GBP, profit: -0.0010589297
T0: HKD, T1: CZK, T2: SGD, profit: -0.0008506308
T0: HKD, T1: CZK, T2

T0: JPY, T1: CAD, T2: NZD, profit: -0.0003940643
T0: JPY, T1: CAD, T2: CHF, profit: -0.0004061254
T0: JPY, T1: CAD, T2: PLN, profit: -0.0011249386
T0: JPY, T1: CAD, T2: CNH, profit: -0.0052036493
T0: JPY, T1: CAD, T2: HUF, profit: -0.0014971556
T0: JPY, T1: CAD, T2: AUD, profit: -0.0003041683
T0: JPY, T1: CAD, T2: SAR, profit: -0.0007871792
T0: JPY, T1: CAD, T2: THB, profit: -0.0009929507
T0: JPY, T1: CAD, T2: TRY, profit: -0.0030070840
T0: JPY, T1: CAD, T2: INR, profit: -0.0010220373
T0: JPY, T1: CAD, T2: DKK, profit: -0.0004641677
T0: JPY, T1: CAD, T2: NOK, profit: -0.0006261535
T0: JPY, T1: CAD, T2: CZK, profit: -0.0011672326
T0: JPY, T1: CAD, T2: MXN, profit: -0.0004714304
T0: JPY, T1: CHF, T2: USD, profit: -0.0001993173
T0: JPY, T1: CHF, T2: ZAR, profit: -0.0014771993
T0: JPY, T1: CHF, T2: EUR, profit: -0.0002746991
T0: JPY, T1: CHF, T2: GBP, profit: -0.0002541714
T0: JPY, T1: CHF, T2: SEK, profit: -0.0006339583
T0: JPY, T1: CHF, T2: SGD, profit: -0.0004193704
T0: JPY, T1: CHF, T2

T0: NZD, T1: SGD, T2: USD, profit: -0.0003699872
T0: NZD, T1: SGD, T2: EUR, profit: -0.0003575159
T0: NZD, T1: SGD, T2: GBP, profit: -0.0004091559
T0: NZD, T1: SGD, T2: HKD, profit: -0.0004304866
T0: NZD, T1: SGD, T2: JPY, profit: -0.0004361412
T0: NZD, T1: SGD, T2: CAD, profit: -0.0004758248
T0: NZD, T1: SGD, T2: CHF, profit: -0.0005668324
T0: NZD, T1: SGD, T2: AUD, profit: -0.0004944424
T0: NZD, T1: HKD, T2: USD, profit: -0.0002920137
T0: NZD, T1: HKD, T2: EUR, profit: -0.0003161067
T0: NZD, T1: HKD, T2: GBP, profit: -0.0003911265
T0: NZD, T1: HKD, T2: SGD, profit: -0.0003720031
T0: NZD, T1: HKD, T2: JPY, profit: -0.0003429851
T0: NZD, T1: HKD, T2: CAD, profit: -0.0004285675
T0: NZD, T1: HKD, T2: CHF, profit: -0.0004639838
T0: NZD, T1: HKD, T2: AUD, profit: -0.0003579822
T0: NZD, T1: JPY, T2: USD, profit: -0.0002697832
T0: NZD, T1: JPY, T2: EUR, profit: -0.0002395296
T0: NZD, T1: JPY, T2: GBP, profit: -0.0003230734
T0: NZD, T1: JPY, T2: SGD, profit: -0.0004204014
T0: NZD, T1: JPY, T2

T0: CAD, T1: HKD, T2: MXN, profit: -0.0004304307
T0: CAD, T1: JPY, T2: USD, profit: -0.0002035392
T0: CAD, T1: JPY, T2: ZAR, profit: -0.0012103728
T0: CAD, T1: JPY, T2: EUR, profit: -0.0001936633
T0: CAD, T1: JPY, T2: GBP, profit: -0.0002472350
T0: CAD, T1: JPY, T2: SEK, profit: -0.0005410718
T0: CAD, T1: JPY, T2: SGD, profit: -0.0003299125
T0: CAD, T1: JPY, T2: HKD, profit: -0.0002461328
T0: CAD, T1: JPY, T2: NZD, profit: -0.0003439383
T0: CAD, T1: JPY, T2: CHF, profit: -0.0003402680
T0: CAD, T1: JPY, T2: PLN, profit: -0.0006988508
T0: CAD, T1: JPY, T2: CNH, profit: -0.0112677711
T0: CAD, T1: JPY, T2: HUF, profit: -0.0012677922
T0: CAD, T1: JPY, T2: AUD, profit: -0.0002621888
T0: CAD, T1: JPY, T2: SAR, profit: -0.0007905238
T0: CAD, T1: JPY, T2: THB, profit: -0.0031173164
T0: CAD, T1: JPY, T2: TRY, profit: -0.0025517806
T0: CAD, T1: JPY, T2: INR, profit: -0.0010065865
T0: CAD, T1: JPY, T2: DKK, profit: -0.0003861971
T0: CAD, T1: JPY, T2: NOK, profit: -0.0004540782
T0: CAD, T1: JPY, T2

limit_output extension: Maximum message size of 100000 exceeded with 100009 characters

In [285]:
abiPair.to_csv('abiPair.csv', index=False)

In [283]:
abiPair.sort_values(by='profit(T0)', ascending=False)

,T0,T1,T2,side1,side2,side3,profit(T0)
1573,JPY,EUR,USD,buy,sell,sell,-0.000026
357,EUR,USD,JPY,sell,sell,buy,-0.000058
1545,JPY,USD,EUR,buy,buy,sell,-0.000070
101,USD,JPY,EUR,sell,buy,sell,-0.000072
452,EUR,JPY,USD,sell,buy,buy,-0.000080
102,USD,JPY,GBP,sell,buy,sell,-0.000084
13,USD,EUR,JPY,buy,sell,buy,-0.000087
1594,JPY,GBP,USD,buy,sell,sell,-0.000098
637,GBP,USD,JPY,sell,sell,buy,-0.000101
1546,JPY,USD,GBP,buy,buy,sell,-0.000102


In [26]:
pip_spread('USD_CHF', print_=True)

0.0001CHF = 1pips, spread: 1.6pips, 0.00016CHF


In [121]:
pip_spread('EUR_USD')[2]

5e-05

In [103]:
T0, T1, T2 = 'USD', 'EUR', 'GBP'

('USD', 'ZAR', 'EUR')

In [29]:
T1_price, T2_price, T0_price

(14.88341, 0.9695, 14.42089)

In [23]:
T0_price - T1_price * T2_price

-824.666602

In [68]:
1 / T0_price * T1_price / T2_price

0.9997577004320077

In [103]:
pip_spread('USD_CNH')

(0.0001, 17.1, 0.0017100000000000001)

In [18]:
params = {"instruments": "USD_CAD"}
pi = PricingInfo(accountID=accountID, params=params)
r = api.request(pi)
T0_price = float(r['prices'][0]['asks'][0]['price'])
params = {"instruments": "USD_JPY"}
pi = PricingInfo(accountID=accountID, params=params)
r = api.request(pi)
T1_price = float(r['prices'][0]['bids'][0]['price'])
params = {"instruments": "CAD_JPY"}
pi = PricingInfo(accountID=accountID, params=params)
r = api.request(pi)
T2_price = float(r['prices'][0]['asks'][0]['price'])
1 / T0_price * T1_price / T2_price

0.9990086528872185

In [288]:
abiPair.sort_values(by='profit(T0)', ascending=False).head()

,T0,T1,T2,side1,side2,side3,profit(T0)
1573,JPY,EUR,USD,buy,sell,sell,-0.000026
357,EUR,USD,JPY,sell,sell,buy,-0.000058
1545,JPY,USD,EUR,buy,buy,sell,-0.000070
101,USD,JPY,EUR,sell,buy,sell,-0.000072
452,EUR,JPY,USD,sell,buy,buy,-0.000080


## プラスになる頻度を調べる

In [12]:
abiPair = pd.read_csv('abiPair.csv')

In [13]:
abi_check = pd.DataFrame(columns=['Time', 'T0', "T1", "T2", "profit(T0)"])
abi_check.to_csv('abi_check.csv', index=False)

In [27]:
f = True
while f:
    try:
        abi_check = pd.read_csv('abi_check.csv')
        j = 0
        time.sleep(5)
        now = datetime.datetime.now()
        abi_tmp = abiPair.sort_values(by='profit(T0)', ascending=False).reset_index(drop=True)
        profit = 1
        for i in range(3):
            if abi_tmp.iat[j, i+3] == 'buy':
                params = {"instruments": "{}_{}".format(abi_tmp.iat[j, (i+1)%3], abi_tmp.iat[j, i])}
                pi = PricingInfo(accountID=accountID, params=params)
                r = api.request(pi)
                price = float(r['prices'][0]['asks'][0]['price'])
                profit = profit / price
            else:
                params = {"instruments": "{}_{}".format(abi_tmp.iat[j, i], abi_tmp.iat[j, (i+1)%3])}
                pi = PricingInfo(accountID=accountID, params=params)
                r = api.request(pi)
                price = float(r['prices'][0]['bids'][0]['price'])
                profit = profit * price
        print('{}, {}, {}, {}, '.format(now, abi_tmp.iat[j, 0], abi_tmp.iat[j, 1], abi_tmp.iat[j, 2]), end='')
        print(profit)
        tmp_se = pd.Series([now, abi_tmp.iat[j, 0], abi_tmp.iat[j, 1], abi_tmp.iat[j, 2], profit], index=abi_check.columns)
        abi_check = abi_check.append(tmp_se, ignore_index=True)
        abi_check.to_csv('abi_check.csv', index=False)
    except Exception as e:
        print(e)
        f = False

2020-01-20 16:53:46.948375, JPY, EUR, USD, 0.9999415163837185
2020-01-20 16:53:52.559805, JPY, EUR, USD, 0.9999415163837185
2020-01-20 16:53:58.169180, JPY, EUR, USD, 0.9999415163837185
2020-01-20 16:54:03.790982, JPY, EUR, USD, 0.9999415163837185
2020-01-20 16:54:09.408034, JPY, EUR, USD, 0.9999415163837185
2020-01-20 16:54:15.016290, JPY, EUR, USD, 0.9999415163837185
2020-01-20 16:54:20.630497, JPY, EUR, USD, 0.9999595369196154
2020-01-20 16:54:26.251372, JPY, EUR, USD, 0.9999595369196154
2020-01-20 16:54:31.870479, JPY, EUR, USD, 0.9999268127858364
2020-01-20 16:54:37.485655, JPY, EUR, USD, 0.9999268127858364
2020-01-20 16:54:43.102363, JPY, EUR, USD, 0.9999218372973991
2020-01-20 16:54:48.722592, JPY, EUR, USD, 0.9999258721781749
2020-01-20 16:54:54.336226, JPY, EUR, USD, 0.9999258721781749
2020-01-20 16:54:59.945793, JPY, EUR, USD, 0.9999349518479427
2020-01-20 16:55:05.560967, JPY, EUR, USD, 0.9999439627875009
2020-01-20 16:55:11.169123, JPY, EUR, USD, 0.9999168613204382
2020-01-

2020-01-20 17:06:14.280382, JPY, EUR, USD, 0.9999347017753514
2020-01-20 17:06:19.895094, JPY, EUR, USD, 0.9999528606156843
2020-01-20 17:06:25.510387, JPY, EUR, USD, 0.9999528606156843
2020-01-20 17:06:31.130983, JPY, EUR, USD, 0.9999289416174892
2020-01-20 17:06:36.752062, JPY, EUR, USD, 0.9999509408943201
2020-01-20 17:06:42.367328, JPY, EUR, USD, 0.9999146219642959
2020-01-20 17:06:47.984331, JPY, EUR, USD, 0.9999227408529099
2020-01-20 17:06:53.600248, JPY, EUR, USD, 0.9999473612350307
2020-01-20 17:06:59.215757, JPY, EUR, USD, 0.9999527306360654
2020-01-20 17:07:04.830857, JPY, EUR, USD, 0.999925491884459
2020-01-20 17:07:10.447986, JPY, EUR, USD, 0.999925491884459
2020-01-20 17:07:16.060003, JPY, EUR, USD, 0.999925491884459
2020-01-20 17:07:21.669518, JPY, EUR, USD, 0.9999136609642302
2020-01-20 17:07:27.288041, JPY, EUR, USD, 0.9999524687123997
2020-01-20 17:07:32.904862, JPY, EUR, USD, 0.9999288104962474
2020-01-20 17:07:38.554041, JPY, EUR, USD, 0.9999288104962474
2020-01-20 

2020-01-20 17:18:41.517631, JPY, EUR, USD, 0.9999255609400708
2020-01-20 17:18:47.140587, JPY, EUR, USD, 0.9999385462731777
2020-01-20 17:18:52.756573, JPY, EUR, USD, 0.999929591093283
2020-01-20 17:18:58.377040, JPY, EUR, USD, 0.999929591093283
2020-01-20 17:19:03.991023, JPY, EUR, USD, 0.999929591093283
2020-01-20 17:19:09.610305, JPY, EUR, USD, 0.9999024793090746
2020-01-20 17:19:15.232779, JPY, EUR, USD, 0.9999361148405266
2020-01-20 17:19:20.847269, JPY, EUR, USD, 0.999929591093283
2020-01-20 17:19:26.459294, JPY, EUR, USD, 0.9999205742703943
2020-01-20 17:19:32.077775, JPY, EUR, USD, 0.9999320842679154
2020-01-20 17:19:37.687477, JPY, EUR, USD, 0.9999320842679154
2020-01-20 17:19:43.306418, JPY, EUR, USD, 0.9999320842679154
2020-01-20 17:19:48.926939, JPY, EUR, USD, 0.9999320842679154
2020-01-20 17:19:54.539434, JPY, EUR, USD, 0.9999320842679154
2020-01-20 17:20:00.154525, JPY, EUR, USD, 0.9999320842679154
2020-01-20 17:20:05.767820, JPY, EUR, USD, 0.9999320842679154
2020-01-20 1

2020-01-20 17:31:08.692660, JPY, EUR, USD, 0.9999398133870254
2020-01-20 17:31:14.306643, JPY, EUR, USD, 0.9999398133870254
2020-01-20 17:31:19.923616, JPY, EUR, USD, 0.9999488286761938
2020-01-20 17:31:25.537624, JPY, EUR, USD, 0.9999367418259197
2020-01-20 17:31:31.153574, JPY, EUR, USD, 0.9999573280518544
2020-01-20 17:31:36.871277, JPY, EUR, USD, 0.9999329037326808
2020-01-20 17:31:42.489392, JPY, EUR, USD, 0.9999475452380369
2020-01-20 17:31:48.105368, JPY, EUR, USD, 0.9999475452380369
2020-01-20 17:31:53.729323, JPY, EUR, USD, 0.999926383292142
2020-01-20 17:31:59.354511, JPY, EUR, USD, 0.999926383292142
2020-01-20 17:32:04.972563, JPY, EUR, USD, 0.9999107855348329
2020-01-20 17:32:10.593320, JPY, EUR, USD, 0.9999198631717635
2020-01-20 17:32:16.211624, JPY, EUR, USD, 0.9999256829563681
2020-01-20 17:32:21.833591, JPY, EUR, USD, 0.9999464600227485
2020-01-20 17:32:27.452065, JPY, EUR, USD, 0.9999284322501986
2020-01-20 17:32:33.072032, JPY, EUR, USD, 0.9999409632520806
2020-01-20

2020-01-20 17:43:36.193350, JPY, EUR, USD, 0.9999546361144073
2020-01-20 17:43:41.810326, JPY, EUR, USD, 0.999921904107235
2020-01-20 17:43:47.431335, JPY, EUR, USD, 0.9999129518244829
2020-01-20 17:43:53.042306, JPY, EUR, USD, 0.9999038748271261
2020-01-20 17:43:58.658991, JPY, EUR, USD, 0.9999129518244829
2020-01-20 17:44:04.280952, JPY, EUR, USD, 0.9999039992143962
2020-01-20 17:44:09.901945, JPY, EUR, USD, 0.9999277155366422
2020-01-20 17:44:15.515408, JPY, EUR, USD, 0.9999277155366422
2020-01-20 17:44:21.130387, JPY, EUR, USD, 0.9999277155366422
2020-01-20 17:44:26.751874, JPY, EUR, USD, 0.9999440819032186
2020-01-20 17:44:32.372301, JPY, EUR, USD, 0.9999226614507036
2020-01-20 17:44:37.985795, JPY, EUR, USD, 0.9999195770348243
2020-01-20 17:44:43.602282, JPY, EUR, USD, 0.9999376091991652
2020-01-20 17:44:49.222247, JPY, EUR, USD, 0.9999376091991652
2020-01-20 17:44:54.850191, JPY, EUR, USD, 0.999928653762737
2020-01-20 17:45:00.468190, JPY, EUR, USD, 0.9999319834027616
2020-01-20

KeyboardInterrupt: 

In [ ]:
abi_check.sort_values(by='profit(T0)', ascending=False)